# ML project
TODO:
- More sophisticated graphs
- Test against other models

In [1]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
#from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MultiLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

from model_architecture import tft, training, val_dataloader, train_dataloader, validation, training

import matplotlib.pyplot as plt

Number of parameters in network: 105.6k


/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_forecasting/data/encoders.py:524: UserWarning: scale is below 1e-7 - consider not centering the data or using data with higher variance for numerical stability
  warnings.warn(
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [2]:
# define trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

pl.seed_everything(42)

trainer = pl.Trainer(
    gpus=0,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
)

Global seed set to 42
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [3]:
# Load the model
#savedTft = tft.load_from_checkpoint(torch.load('./app/lightning_logs/version_3/checkpoints/epoch=2-step=90.ckpt'))

In [4]:
# fit the model
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)
#savedTft

/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1     
3  | prescalers                         | ModuleDict                      | 512   
4  | static_variable_selection          | VariableSelectionNetwork        | 2.0 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 32.2 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 16.1 K

Sanity Checking: 0it [00:00, ?it/s]

/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [5]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)

best_model_path = trainer.checkpoint_callback.best_model_path
print(best_model_path)
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
#best_tft = savedTft # TODO: Comment out when retraining again

raw_prediction, x = best_tft.predict(
    #training.filter(lambda x: (x.constant_group == "group_1")),
    training,
    mode="raw",
    return_x=True,
)

/Users/deividas/Workplace/Workforce_Scheduler_Workspace/demand-forecaster-sam/lightning_logs/version_36/checkpoints/epoch=20-step=630.ckpt


/Users/deividas/Library/Python/3.9/lib/python/site-packages/pytorch_forecasting/models/base_model.py:92: UserWarning: Not all dimensions are equal for tensors shapes. Example tensor torch.Size([64, 63, 1, 540]). Returning list instead of torch.Tensor.
  warnings.warn(


In [8]:
# Save the temporal fusion transformer model
torch.save(trainer.checkpoint_callback.best_model_path, './Saved_Models/40MinsTrain_105k_params.pth')
!zip  -r model.zip ./lightning_logs/version_36/*

  adding: lightning_logs/version_36/checkpoints/ (stored 0%)
  adding: lightning_logs/version_36/checkpoints/epoch=20-step=630.ckpt (deflated 25%)
  adding: lightning_logs/version_36/events.out.tfevents.1676992695.Deividass-MacBook-Pro.local (deflated 4%)
  adding: lightning_logs/version_36/hparams.yaml (deflated 93%)


In [ ]:
# calcualte mean absolute error on validation set -> Only works for one output
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean().item()

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True, show_progress_bar=True)
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
#for idx in range(10):  # plot 10 examples 
best_tft.plot_prediction(x, raw_predictions, idx=0)


# plt.show only the first 200 samples
plt.xlim(-100, 80)
plt.show()